In [1]:
!pip install pandasql

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import pytz
from pandasql import sqldf

In [3]:
def utc_to_local(df, col):
    '''
    Convert utc time column into local time, used with lambda
    Accept:
        df: DataFrame
        col: str, utc time column name
    Return: converted local datetime
    '''
    df[col] = dt.datetime.strptime(df[col], '%Y-%m-%d %H:%M:%S')
    tz = pytz.timezone(df['TimeZone'])
    return df[col].replace(tzinfo=pytz.utc).astimezone(tz).replace(tzinfo=None)

In [4]:
def fill_weather_type(df):
    '''
    Fill weather type for every flight, if no special weather return Clear. Used with df.apply()
    Accept:
        df: Row of dataframe
    Return: Corresponding weather type if exists for the flight. 'Clear' if not exist.
    '''
    if df['idx'] in df_flight_weather['idx'].values:
        return df_flight_weather[df_flight_weather['idx']==df['idx']]['dep_weather_type'].unique()[0]
    else:
        return 'Clear'

In [5]:
def fill_weather_sev(df):
    '''
    Fill weather severity for every flight, if no special weather return Clear. Used with df.apply()
    Accept:
        df: Row of dataframe
    Return: Corresponding weather severity if exists for the flight. 'Clear' if not exist.
    '''
    if df['idx'] in df_flight_weather['idx'].values:
        return df_flight_weather[df_flight_weather['idx']==df['idx']]['dep_weather_severity'].unique()[-1]
    else:
        return 'Clear'

In [6]:
df_flight = pd.read_csv('train_flights_Jan_1w.csv') # Read flight data

In [7]:
df_weather = pd.read_csv('WeatherEvents_Aug16_Dec20_Publish.csv') # Read weather data

In [8]:
df_weather = df_weather[df_weather['StartTime(UTC)'].str.startswith('2018')|df_weather['StartTime(UTC)'].str.startswith('2019')] # Narrow down weather data to year 2018/19

In [9]:
# Convert utc time of weather event into local datetime
df_weather['StartTime_local'] = df_weather[['StartTime(UTC)', 'TimeZone']].apply(lambda df: utc_to_local(df, 'StartTime(UTC)'), axis=1)
df_weather['EndTime_local'] = df_weather[['EndTime(UTC)', 'TimeZone']].apply(lambda df: utc_to_local(df, 'EndTime(UTC)'), axis=1)

# Create CRS depature local datetime 
df_flight['dep_datetime'] = (df_flight['fl_date']+' '+ df_flight['crs_dep_time'].apply(lambda x: str(x).zfill(4))).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H%M'))

In [10]:
# Create depature city
df_flight['dep_city'] = df_flight['origin_city_name'].str[:-4]

In [11]:
# Create idx for later weather fill check
df_flight['idx'] = df_flight.index

In [12]:
# Use sql to merge flight and weather data on condition
pysqldf = lambda q: sqldf(q, globals())

In [13]:
q = """
    SELECT a.idx, b.Type AS dep_weather_type, b.Severity AS dep_weather_severity
    FROM df_flight a
    JOIN df_weather b ON a.dep_city = b.City
    WHERE a.dep_datetime BETWEEN b.StartTime_local AND b.EndTime_local;
"""

In [14]:
df_flight_weather = pysqldf(q)

In [15]:
# Drop duplicates
df_flight_weather.drop_duplicates(inplace=True)

In [16]:
df_flight.shape

(224768, 45)

In [17]:
df_flight_weather.shape

(36742, 3)

In [18]:
# Fill weather type and severity to flights
df_flight['dep_weather_type'] = df_flight.apply(fill_weather_type, axis=1)
df_flight['dep_weather_severity'] = df_flight.apply(fill_weather_sev, axis=1)

In [19]:
# Drop intermediate columns
df_flight.drop(columns=['dep_datetime', 'dep_city', 'idx'], inplace=True)

In [20]:
df_flight['dep_weather_type'].apply(lambda x: str(x)).unique()

array(['Clear', 'Rain', 'Fog', 'Precipitation', 'Snow', 'Cold', 'Hail',
       'Storm'], dtype=object)

In [21]:
df_flight['dep_weather_severity'].apply(lambda x: str(x)).unique()

array(['Clear', 'Light', 'Moderate', 'UNK', 'Severe', 'Heavy', 'Other'],
      dtype=object)

In [22]:
df_flight.to_csv('train_flights_weather_Jan_1w.csv')